<h1>Topics<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import pandas as pd 
import plotly.express as px

# Read in the data
diabetes = pd.read_csv('https://bitbucket.org/jimcody/sampledata/raw/b2aa6df015816ec35afc482b53df1b7ca7a31f80/diabetes_for_plotly.csv')

# Clean up the data
diabetes['gender'] = diabetes['gender'].replace({'M':'Male', 'Mle':'Male', 'F':'Female', 
                                                 'female':'Female', 'male':'Male', 
                                                 '?':'Female', 'Unknown/Invalid':'Female'})

diabetes = diabetes.drop('encounter_id',axis=1)
diabetes = diabetes.drop('patient_nbr',axis=1)
diabetes['admission_type_id'] = diabetes.admission_type_id.astype(str)
diabetes['discharge_disposition_id'] = diabetes.discharge_disposition_id.astype(str)

#Create an aggregated dataset (dataframe)
dm = diabetes.groupby(['year','month','gender','admission_type_id','discharge_disposition_id']).sum().reset_index()
dm = dm.sort_values(['year','month'])
#d_month.head()

In [ ]:
dm.info()

# Add dashboard code below

# Dash Exercise 1 - 20 minutes

- Use the dm dataframe.  This has the raw diabetes data aggregated
- In the cell below, add all of your dashboard code.
- The dashboard should include a scatterplot of num_procedures vs num_lab_procedures.
- Add two more plots of any style.

In [ ]:
#!pip install jupyter_dash

In [ ]:
from jupyter_dash import JupyterDash  
from dash.dependencies import Output, Input
from dash import no_update
from dash import dcc
from dash import html

# Adding a Radio Item

In [ ]:
labs = px.scatter(dm, 
                  x='num_lab_procedures', 
                  y='num_procedures',
                  size='num_medications', 
                  color='month', 
                  hover_name='month')

time = px.bar(dm, x="month", y="time_in_hospital")
admin = px.bar(dm, x="admission_type_id", y="num_lab_procedures")

app = JupyterDash(__name__) 

# Add dcc.Graph code here
app.layout = html.Div([
    
    html.Div([dcc.Graph(id='x', figure=labs )]),
    'Admin Type: ',
    html.Div([dcc.RadioItems(id='s-year',
                             options=[2019,2020,2021], 
                             value=2019)])
])

#app.run_server(mode='inline')
app.run_server(mode='external', port = 8074)

# Adding the callback & reusable component

In [103]:
labs = px.scatter(dm, 
                  x='num_lab_procedures', 
                  y='num_procedures',
                  size='num_medications', 
                  color='month', 
                  hover_name='month')

app = JupyterDash(__name__) 

app.layout = html.Div([
    
    dcc.Graph(id='fig1'),
    'Select a Year:  ',
    html.P(),
    dcc.RadioItems(id='year-s', options=[2019,2020,2021], value=2019)  ])

@app.callback(
    Output('fig1','figure'),               
    Input('year-s','value'))

def update_figure(xyz):
    filtered_df = dm[dm.year == xyz]

    labs = px.scatter(filtered_df, 
                     x='num_lab_procedures', 
                     y='num_procedures')

    labs.update_layout(transition_duration=500)

    return labs
    
#app.run_server(mode='inline')
app.run_server(mode='external', port = 8073)

Dash app running on http://127.0.0.1:8073/


In [ ]:
# Output & Input are the arguements of the callback decorator.
# properties of a particular component
# Our input is the "value" property of the component that has the ID 'admintype'. 
# Our output is the "children" property of the component with the ID 'x'.
# There are only 2 keyword arguements - component_id and component_property.  Can be omitted.